In [1]:
vendedores=spark.read.csv("vendedores.csv",sep=";",
                          header=True,inferSchema=True)

In [2]:
vendedores.show()

+------+--------+------+
|Nombre| Empresa|Ventas|
+------+--------+------+
| Pedro|  Google|100000|
| María|Facebook|200000|
| Marta|    null|150000|
|   Ana|  Google|  null|
|  null|Facebook|150000|
+------+--------+------+



In [4]:
vendedores.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Empresa: string (nullable = true)
 |-- Ventas: integer (nullable = true)



In [5]:
vendedores.summary().show()

+-------+------+--------+----------------+
|summary|Nombre| Empresa|          Ventas|
+-------+------+--------+----------------+
|  count|     4|       4|               4|
|   mean|  null|    null|        150000.0|
| stddev|  null|    null|40824.8290463863|
|    min|   Ana|Facebook|          100000|
|    25%|  null|    null|          100000|
|    50%|  null|    null|          150000|
|    75%|  null|    null|          150000|
|    max| Pedro|  Google|          200000|
+-------+------+--------+----------------+



In [6]:
empresas=vendedores.groupBy("Empresa")

In [7]:
empresas.sum().show()

+--------+-----------+
| Empresa|sum(Ventas)|
+--------+-----------+
|    null|     150000|
|  Google|     100000|
|Facebook|     350000|
+--------+-----------+



In [9]:
from pyspark.sql.functions import sum,avg,stddev,count
ventas=vendedores.Ventas
vendedores.select(sum(ventas),avg(ventas),stddev(ventas)).show()

+-----------+-----------+-------------------+
|sum(Ventas)|avg(Ventas)|stddev_samp(Ventas)|
+-----------+-----------+-------------------+
|     600000|   150000.0|   40824.8290463863|
+-----------+-----------+-------------------+



In [18]:
nuevoVendedores=vendedores.na.drop(how="any",subset=["Ventas"
                                           ,"Empresa"])
nuevoVendedores.show()

+------+--------+------+
|Nombre| Empresa|Ventas|
+------+--------+------+
| Pedro|  Google|100000|
| María|Facebook|200000|
|  null|Facebook|150000|
+------+--------+------+



In [21]:
nuevoVendedores=vendedores.na.drop(thresh=3)
nuevoVendedores.show()

+------+--------+------+
|Nombre| Empresa|Ventas|
+------+--------+------+
| Pedro|  Google|100000|
| María|Facebook|200000|
+------+--------+------+



In [26]:
nuevoVendedores=vendedores.na.fill("Vendedor A",
    subset=["Nombre"]).na.fill("Empresa 1",
    subset=["Empresa"]).na.fill(0,subset=["Ventas"])
nuevoVendedores.show()

+----------+---------+------+
|    Nombre|  Empresa|Ventas|
+----------+---------+------+
|     Pedro|   Google|100000|
|     María| Facebook|200000|
|     Marta|Empresa 1|150000|
|       Ana|   Google|     0|
|Vendedor A| Facebook|150000|
+----------+---------+------+



In [41]:
empresas=["Google","Facebook"]
empresasAnonimas=["Empresa 2","Empresa 3"]
anonima=nuevoVendedores.replace(empresas,
    empresasAnonimas,"Empresa").replace(["Pedro",
    "María","Marta","Ana"],["Vendedor B","Vendedor C",
    "Vendedor D","Vendedor E"],"Nombre")
anonima.show()

+----------+---------+------+
|    Nombre|  Empresa|Ventas|
+----------+---------+------+
|Vendedor B|Empresa 2|100000|
|Vendedor C|Empresa 3|200000|
|Vendedor D|Empresa 1|150000|
|Vendedor E|Empresa 2|     0|
|Vendedor A|Empresa 3|150000|
+----------+---------+------+

